# Работа с получением данных из сервиса EDGAR SEC

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
%matplotlib inline

from pathlib import Path
from datetime import date
import json
from io import BytesIO
from zipfile import ZipFile, BadZipFile
import requests
import os

import pandas_datareader.data as web
import pandas as pd

from pprint import pprint

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [2]:
sns.set_style('whitegrid')

In [3]:
data_path = Path('data') # perhaps set to external harddrive to accomodate large amount of data
if not data_path.exists():
    data_path.mkdir()

## Загрузка финасовых документов и заметок 

Данные с сайта [Financial Statement and Notes](https://www.sec.gov/dera/data/financial-statement-and-notes-data-set.html)

In [4]:
SEC_URL = 'https://www.sec.gov/'
FSN_PATH = 'files/dera/data/financial-statement-and-notes-data-sets/'
start_year = 2015

In [5]:
today = pd.Timestamp(date.today())
this_year = today.year
this_quarter = today.quarter
past_years = range(start_year, this_year)

In [6]:
filing_periods  = [(y, q)for y in past_years for q in range(1, 5)]
filing_periods.extend([this_year, q] for q in range(1, this_quarter+1))

In [54]:
for i, (y, q) in enumerate(filing_periods):
    print(f'{y}-Q{q}', end=' \n', flush=True)
    filing = f'{y}q{q}_notes.zip'
    path = data_path / f'{y}_{q}' / 'source'
    if not path.exists():
        path.mkdir(exist_ok=True, parents=True)
    url = SEC_URL + FSN_PATH + filing
    
    response = requests.get(url).content
    try:
        with ZipFile(BytesIO(response)) as zip_file:
            zip_file.extractall(path)
#             for file in zip_file.namelist():
#                 local_file = path / file
#                 if local_file.exists():
#                     continue
#                     try:
#                         with local_file.open('wb').readlines as output:
#                             for line in zip_file.open(file).readlines:
#                                 output.write(line)
#                     except Exception as e:
#                         print(e)
#                         continue
    except BadZipFile:
        'https://www.sec.gov/files/node/add/data_distribution/2020q1_notes.zip'
        print('got bad zip file')
        continue

2015-Q1 
2015-Q2 
2015-Q3 
2015-Q4 
2016-Q1 
2016-Q2 
2016-Q3 
2016-Q4 
2017-Q1 
2017-Q2 
2017-Q3 
2017-Q4 
2018-Q1 
2018-Q2 
2018-Q3 
2018-Q4 
2019-Q1 
2019-Q2 
2019-Q3 
2019-Q4 
2020-Q1 
2020-Q2 
2020-Q3 
2020-Q4 
2021-Q1 


BadZipFile: File is not a zip file

## Сохранение в parquet

Исходные текстовые текстовые файл занимают довольно большой объем пространства и для более быстрого доступа к данным лучше преобразовать текстовые файлы в двоичный формат

In [8]:
for f in data_path.glob('**/*.tsv'):
    file_name = f.stem + '.parquet'
    path = Path(f.parents[1]) / 'parquet'
    if (path / file_name).exists():
        continue
    if not path.exists():
        path.mkdir(exist_ok=True)
    try:
        df = pd.read_csv(f, sep='\t', encoding='latin1', low_memory=False)
    except:
        print(f)
    df.to_parquet(path / file_name)

data/2017_4/source/tag.tsv
data/2019_2/source/tag.tsv
data/2019_3/source/tag.tsv
data/2020_1/source/tag.tsv
data/2020_2/source/tag.tsv
data/2020_3/source/tag.tsv


## Организация данных

Данные разбиты на наборы ежеквартальных документов, содежащих информацию о поданной отчетности, тегах таксономии и т.п.

| Имя файла | Dataset      | Описание                                                 |
|------|--------------|-------------------------------------------------------------|
| SUB  | Submission   | Идентифицирует XBRL представление компании, форме, дате и т.д. |
| TAG  | Tag          | Определяет и объясняет каждый тэг таксономии                      |
| DIM  | Dimension    | Добавляет детальные численные и текстовые данные                  |
| NUM  | Numeric      | Одна строка для каждой отдельной точки данных в архиве              |
| TXT  | Plain Text   | Содержит все нечисловые поля XBRL                        |
| REN  | Rendering    | Информация для рендеринга на сайте SEC                    |
| PRE  | Presentation | Подробная информация о представлении тегов и номеров в первичных документах |
| CAL  | Calculation  | Показывает арифметические зависимости между тегами                   |

## Идентификационные данные

Данные содержат уникальные идентикаторы, необходимые для получения документов: центральный индексный ключ (cik) и регистрационный номер (adsh).

In [106]:
sub = pd.read_parquet(data_path / '2020_4' / 'parquet' / 'sub.parquet')
sub.name = sub.name.apply(lambda x: x.lower())

In [122]:
name = 'tesla'
form = '10-Q'
firm = sub[sub.name.apply(lambda x: x.find(name.lower()) != -1)]
firm = firm[firm.form == form]
key_cols = ['name', 'adsh', 'cik', 'name', 'sic', 'countryba', 'stprba',
            'cityba', 'zipba', 'bas1', 'form', 'period', 'fy', 'fp', 'filed']
firm = firm[key_cols]
firm = firm.T.dropna().squeeze().drop_duplicates()
firm

name                  tesla, inc.
adsh         0001564590-20-047486
cik                       1318605
sic                          3711
countryba                      US
stprba                         CA
cityba                  PALO ALTO
zipba                       94304
bas1           3500 DEER CREEK RD
form                         10-Q
period                   20200930
fy                           2020
fp                             Q3
filed                    20201026
Name: 2715, dtype: object

##  Создание набора данных на основании идентификационных данных

Используя индентификационных ключ cik, получаем все исторические данные по отчетам, доступным для компании.

### Получение документов

In [123]:
firm_subs = pd.DataFrame()
for sub in data_path.glob('**/sub.parquet'):
    sub = pd.read_parquet(sub)
    firm_sub = sub[(sub.cik.astype(int) == firm.cik) & (sub.form.isin(['10-Q', '10-K']))]
    firm_subs = pd.concat([firm_subs, firm_sub])

In [125]:
firm_subs.form.value_counts()

10-Q    21
10-K     7
Name: form, dtype: int64